Assignment I: Exploratory Data Analysis

1. Import the dataset into a pandas dataframe. Make sure that the date column is
in pandas date time format.

In [2]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv("/content/train.csv")

# Convert to datetime format, specifying the correct format
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M')

df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


2. Check the data type of each column. How many rows are there in the dataset
? Does the dataset contain any missing values ?

In [3]:
print("DATA TYPES:\n", df.dtypes)
print("\nNumber of rows:", df.shape[0])
print("\nMissing values per column:\n", df.isnull().sum())


DATA TYPES:
 datetime      datetime64[ns]
season                 int64
holiday                int64
workingday             int64
weather                int64
temp                 float64
atemp                float64
humidity               int64
windspeed            float64
casual                 int64
registered             int64
count                  int64
dtype: object

Number of rows: 10886

Missing values per column:
 datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
casual        0
registered    0
count         0
dtype: int64


3. Using the date column, create new columns for: year, month, day of the week
and hour of the day.

In [4]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

df[['year','month','dayofweek','hour']].head()


,year,month,dayofweek,hour
0,2011,1,5,0
1,2011,1,5,1
2,2011,1,5,2
3,2011,1,5,3
4,2011,1,5,4


4. Rename the values in the season column to spring, summer, fall and winter

In [5]:
season_map = {1: "spring", 2: "summer", 3: "fall", 4: "winter"}
df['season'] = df['season'].map(season_map)

df['season'].value_counts()


,count
season,
winter,2734
summer,2733
fall,2733
spring,2686


5. Calculate the total number of casual and registered bikes rented in the years
2011 and 2012.

In [6]:
totals_by_year = df.groupby('year')[['casual','registered']].sum()
totals_by_year


,casual,registered
year,,
2011,155817,626162
2012,236318,1067179


6. Calculate the mean of the hourly total rentals count by season. Which season
has the highest mean ?

In [7]:
mean_count_by_season = df.groupby('season')['count'].mean().sort_values(ascending=False)
mean_count_by_season


,count
season,
fall,234.417124
summer,215.251372
winter,198.988296
spring,116.343261


7. Are more bikes rented by registered users on working or non-working days ?  
Does the answer differ for non-registered users ? Is the answer the same for
both years ?

In [8]:
# Overall means
registered_working = df.groupby('workingday')['registered'].mean()
casual_working = df.groupby('workingday')['casual'].mean()

print("Registered users (0=non-working, 1=working):\n", registered_working)
print("\nCasual users (0=non-working, 1=working):\n", casual_working)

# By year
registered_by_year = df.groupby(['year','workingday'])['registered'].mean().unstack()
casual_by_year = df.groupby(['year','workingday'])['casual'].mean().unstack()

print("\nRegistered rentals by year:\n", registered_by_year)
print("\nCasual rentals by year:\n", casual_by_year)


Registered users (0=non-working, 1=working):
 workingday
0    129.198330
1    167.904209
Name: registered, dtype: float64

Casual users (0=non-working, 1=working):
 workingday
0    59.308290
1    25.107663
Name: casual, dtype: float64

Registered rentals by year:
 workingday           0           1
year                              
2011         97.153132  124.031909
2012        160.767429  211.587507

Casual rentals by year:
 workingday          0          1
year                            
2011        48.201276  19.664143
2012        70.250286  30.527733


8. Which months in the year 2011 have the highest and the lowest total number of
bikes rented ? Repeat for the year 2012.

In [9]:
monthly_totals = df.groupby(['year','month'])['count'].sum()

print("2011 highest month:", monthly_totals.loc[2011].idxmax())
print("2011 lowest month:", monthly_totals.loc[2011].idxmin())

print("2012 highest month:", monthly_totals.loc[2012].idxmax())
print("2012 lowest month:", monthly_totals.loc[2012].idxmin())


2011 highest month: 7
2011 lowest month: 1
2012 highest month: 9
2012 lowest month: 1


9. Which type of weather have the highest and lowest mean of the hourly total
rentals count ?

In [10]:
mean_by_weather = df.groupby('weather')['count'].mean().sort_values(ascending=False)
mean_by_weather


,count
weather,
1,205.236791
2,178.955540
4,164.000000
3,118.846333


10. Calculate the correlation between the hourly total rentals count and all the
numerical columns in the dataset. Which column has the highest correlation
with the total rentals count ?

In [11]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_with_count = df[numeric_cols].corr()['count'].sort_values(ascending=False)
corr_with_count


,count
count,1.000000
registered,0.970948
casual,0.690414
hour,0.400601
temp,0.394454
atemp,0.389784
year,0.260403
month,0.166862
windspeed,0.101369
workingday,0.011594


11. Create a new categorical column called day_period, which can take four
possible values: night, morning, afternoon and evening. These values
correspond to the following binning of the hour column: 06 night, 612
morning, 126 afternoon, 624:evening.

In [12]:
bins = [0, 6, 12, 18, 24]
labels = ['night','morning','afternoon','evening']

df['day_period'] = pd.cut(
    df['hour'],
    bins=bins,
    labels=labels,
    right=False,
    include_lowest=True
)

df[['hour','day_period']].head(20)


,hour,day_period
0,0,night
1,1,night
2,2,night
3,3,night
4,4,night
5,5,night
6,6,morning
7,7,morning
8,8,morning
9,9,morning


12. Generate a pivot table for the mean of the hourly total rentals count, with the
index set to the day period and the column set to the working day column.
What can you observe from the table ?

In [13]:
pivot = df.pivot_table(
    values='count',
    index='day_period',
    columns='workingday',
    aggfunc='mean'
)

pivot


/tmp/ipython-input-3082248797.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot = df.pivot_table(


workingday,0,1
day_period,,
night,44.052083,16.217582
morning,157.772414,234.353763
afternoon,371.022989,266.196141
evening,180.182759,251.054662
